# Introduction

Analyse d'une option de contrôles réalisés sur la chaîne de caractère Editeur pour valider une proposition d'alignement réalisée par Bibliostratus quand l'alignement est fait
* à partir de l'ISBN
* à partir d'une recherche Titre-Auteur-Date

4 valeurs sont prévues dans les préférences :
* 0 : pas de contrôle
* 1 : la valeur de l'éditeur d'une notice est présente dans l'autre (ou inversement), sauf si vide (en ce cas, pas de contrôle)
* 2 : la valeur de l'éditeur d'une notice est présente dans l'autre. Si un éditeur est vide et l'autre renseigné, l'alignement est annulé
* 3 : valeurs strictement identiques (toutes 2 renseignées ou toutes 2 vides)

In [136]:
from stdf import *
from collections import Counter, defaultdict
import pandas as pd

In [147]:
file_pref0 = file2list("controle_editeur_pref0-resultats_bib2id.txt", all_cols=True)
file_pref1 = file2list("controle_editeur_pref1-resultats_bib2id.txt", all_cols=True)
file_pref2 = file2list("controle_editeur_pref2-resultats_bib2id.txt", all_cols=True)
file_pref3 = file2list("controle_editeur_pref3-resultats_bib2id.txt", all_cols=True)

headers = file_pref0.pop(0)
file_pref1.pop(0)
file_pref2.pop(0)
file_pref3.pop(0)
headers.insert(0, "N°")

In [148]:
for i in range(len(headers)):
    print(i, headers[i])

0 N°
1 NumNot
2 Nb identifiants trouvés
3 Liste identifiants trouvés
4 Méthode d'alignement
5 FRBNF
6 ARK
7 ISBN
8 EAN
9 Titre
10 Auteur
11 Date
12 Volume-Tome
13 Editeur
14 Type doc / Type notice
15 [BnF/Abes] ISBN
16 [BnF/Abes] EAN
17 [BnF/Abes] Référence commerciale
18 [BnF/Abes] ISSN
19 [BnF/Abes] Titre
20 [BnF/Abes] Auteur(s)
21 [BnF/Abes] Date
22 [BnF/Abes] Tome/Volume
23 [BnF/Abes] Editeur
24 [BnF/Abes] Lieu de publication
25 [BnF/Abes] Ids pérennes BnF/Abes
26 [BnF/Abes] Type notice-doct


In [149]:
for i in range(len(file_pref0)):
    file_pref0[i].insert(0, str(i))
    file_pref1[i].insert(0, str(i))
    file_pref2[i].insert(0, str(i))
    file_pref3[i].insert(0, str(i))
    

In [150]:
class Record:
    def __init__(self, row):
        self.id = row[0]
        self.recordid = row[1]
        self.nb_arks = row[2]
        self.arks = row[3]
        self.init_publisher = row[13]
        try:
            self.found_publisher = row[22]
        except IndexError:
            self.found_publisher = ""

class Records_pack:
    def __init__(self, rec0, rec1, rec2, rec3):
        self.id = rec0.id
        self.rec0 = rec0
        self.rec1 = rec1
        self.rec2 = rec2
        self.rec3 = rec3
        self.nb_arks = [rec0.nb_arks, rec1.nb_arks, rec2.nb_arks, rec3.nb_arks]
        self.arks_diff = rec0.nb_arks == rec1.nb_arks == rec2.nb_arks == rec3.nb_arks
        self.arks0_1_diff = rec0.nb_arks != rec1.nb_arks
        self.arks1_2_diff = rec1.nb_arks != rec2.nb_arks
        self.arks2_3_diff = rec2.nb_arks != rec3.nb_arks

In [151]:
records = []
for i in range(len(file_pref0)):
    rec0 = Record(file_pref0[i])
    rec1 = Record(file_pref1[i])
    rec2 = Record(file_pref2[i])
    rec3 = Record(file_pref3[i])
    if rec0.id == rec1.id == rec2.id == rec3.id:
        records.append(Records_pack(rec0, rec1, rec2, rec3))

In [152]:
print(records[8].rec0.init_publisher)

Amboise Bray libraire-éditeur


In [153]:
rec_pareils = []
rec_diff_0_1 = []
rec_diff_1_2 = []
rec_diff_2_3 = []

In [154]:
for rec in records:
    if rec.arks_diff:
        if  rec.rec0.nb_arks != "0":
            rec_pareils.append(rec)
    else:
        if rec.arks0_1_diff:
            rec_diff_0_1.append(rec)
        if rec.arks1_2_diff:
            rec_diff_1_2.append(rec)
        if rec.arks2_3_diff:
            rec_diff_2_3.append(rec)

In [155]:
print("rec_pareils", len(rec_pareils))
print("rec_diff_0_1", len(rec_diff_0_1))
print("rec_diff_1_2", len(rec_diff_1_2))
print("rec_diff_2_3", len(rec_diff_2_3))

rec_pareils 270
rec_diff_0_1 47
rec_diff_1_2 29
rec_diff_2_3 60


## Echantillon

In [162]:
def get_echantillon(liste):
    df = []
    for rec in liste[0:20]:
        local = {"id": rec.id, "recnum": rec.rec0.recordid, "editeur init": rec.rec0.init_publisher, 
                 "c0 - nbARK": rec.rec0.nb_arks, "c0 - éditeurs": rec.rec0.found_publisher,
                 "c1 - nbARK": rec.rec1.nb_arks, "c1 - éditeurs": rec.rec1.found_publisher,
                 "c2 - nbARK": rec.rec2.nb_arks, "c2 - éditeurs": rec.rec2.found_publisher,
                 "c3 - nbARK": rec.rec3.nb_arks, "c3 - éditeurs": rec.rec3.found_publisher}
        df.append(local)
    df = pd.DataFrame(df)
    return df

### Notices avec résultats identiques

In [163]:
rec_pareils_echantillon = get_echantillon(rec_pareils)
rec_pareils_echantillon

,id,recnum,editeur init,c0 - nbARK,c0 - éditeurs,c1 - nbARK,c1 - éditeurs,c2 - nbARK,c2 - éditeurs,c3 - nbARK,c3 - éditeurs
0,14,DOB00002341,Archaeopress,1,Archaeopress,1,Archaeopress,1,Archaeopress,1,Archaeopress
1,49,ARC00006337,Saint-Nazaire Histoire,1,,1,,1,,1,
2,50,FRBNF456152380000004,,1,,1,,1,,1,
3,57,DOB00002278,Olivares,1,Olivares,1,Olivares,1,Olivares,1,Olivares
4,72,DOB00002348,Editions Monique Mergoil,1,,1,,1,,1,
5,79,FRBNF457338620000003,Casterman,1,,1,,1,,1,
6,81,DOB00002293,Ed du Patrimoine Centre des monuments nationaux,1,Éd du Patrimoine Centre des monuments nationaux,1,Éd du Patrimoine Centre des monuments nationaux,1,Éd du Patrimoine Centre des monuments nationaux,1,Éd du Patrimoine Centre des monuments nationaux
7,83,ARC00006408,Presses universitaires de Limoges,1,Pulim,1,Pulim,1,Pulim,1,Pulim
8,84,ARC00006407,Presses universitaires de Limoges,1,Pulim,1,Pulim,1,Pulim,1,Pulim
9,85,DOB00002272,Editions Snoeck,1,,1,,1,,1,


### Notices dont le nombre de résultats diffère quand on passe du niveau de contrôle de 0 à 1

In [164]:
rec_diff_0_1_echantillon = get_echantillon(rec_diff_0_1)
rec_diff_0_1_echantillon

,id,recnum,editeur init,c0 - nbARK,c0 - éditeurs,c1 - nbARK,c1 - éditeurs,c2 - nbARK,c2 - éditeurs,c3 - nbARK,c3 - éditeurs
0,42,ARC00006987,Imprimerie Alpha,1,,0,,0,,0,
1,45,ARC00007111,Presses universitaires de Rennes,1,,0,,0,,0,
2,47,ARC00006348,Au bord du fleuve,1,,0,,0,,0,
3,58,DOB00002352,Bibliothèque municipale de Rennes,1,Bibliothèque municipale,0,,0,,0,
4,80,ARC00006488,Honoré et Edouard Champion,1,H et E Champion,0,,0,,0,
5,126,ARC00006532,Ernest Flammarion,23,E Flammarion|E Flammarion|Vision nouvelle|Le...,10,E Flammarion|E Flammarion|Flammarion|E Flam...,8,E Flammarion|E Flammarion|Flammarion|E Flam...,0,
6,210,ARC00007121,Editions 303,1,,0,,0,,0,
7,317,ARC00006915,Editions Ouest-France,1,,0,,0,,0,
8,345,DOB00002356,Bibliothèque Nationale,1,Crédit lyonnais,0,,0,,0,
9,433,ARC00006620,Editions du Seuil,2,|,0,,0,,0,


### Notices dont le nombre de résultats diffère quand on passe du niveau de contrôle de 1 à 2

In [165]:
rec_diff_1_2_echantillon = get_echantillon(rec_diff_1_2)
rec_diff_1_2_echantillon

,id,recnum,editeur init,c0 - nbARK,c0 - éditeurs,c1 - nbARK,c1 - éditeurs,c2 - nbARK,c2 - éditeurs,c3 - nbARK,c3 - éditeurs
0,77,ARC00006493,Moutard,2,|,2,|,0,,0,
1,78,ARC00006492,Moutard,2,|,2,|,0,,0,
2,126,ARC00006532,Ernest Flammarion,23,E Flammarion|E Flammarion|Vision nouvelle|Le...,10,E Flammarion|E Flammarion|Flammarion|E Flam...,8,E Flammarion|E Flammarion|Flammarion|E Flam...,0,
3,165,ARC00006655,,1,,1,,0,,0,
4,173,ARC00007008,Georges Ondet,1,,1,,0,,0,
5,180,ARC00006475,Calmann-Lévy,2,C Lévy|,2,C Lévy|,1,C Lévy,0,
6,239,ARC00006576,Chez Verdière,1,,1,,0,,0,
7,252,DOB00002351,,1,Les bibliolâtres de France,1,Les bibliolâtres de France,0,,0,
8,258,ARC00007124,,374,||impr de E Chaulmontet|||Thunot et Ce|C Sa...,374,||impr de E Chaulmontet|||Thunot et Ce|C Sa...,51,||||||||||||||||||||||||||||||||||||||||||||||...,51,||||||||||||||||||||||||||||||||||||||||||||||...
9,368,ARC00006403,,1,Revue 303,1,Revue 303,0,,0,


### Notices dont le nombre de résultats diffère quand on passe du niveau de contrôle de 2 à 3

In [166]:
rec_diff_2_3_echantillon = get_echantillon(rec_diff_2_3)
rec_diff_2_3_echantillon

,id,recnum,editeur init,c0 - nbARK,c0 - éditeurs,c1 - nbARK,c1 - éditeurs,c2 - nbARK,c2 - éditeurs,c3 - nbARK,c3 - éditeurs
0,8,ARC00006825,Amboise Bray libraire-éditeur,1,A Bray,1,A Bray,1,A Bray,0,
1,73,DOB00002385,British Library,1,The British Library,1,The British Library,1,The British Library,0,
2,112,ARC00006612,Editions du Seuil,1,Seuil,1,Seuil,1,Seuil,0,
3,126,ARC00006532,Ernest Flammarion,23,E Flammarion|E Flammarion|Vision nouvelle|Le...,10,E Flammarion|E Flammarion|Flammarion|E Flam...,8,E Flammarion|E Flammarion|Flammarion|E Flam...,0,
4,154,ARC00007016,Librairie Académique Perrin et Cie,1,Perrin,1,Perrin,1,Perrin,0,
5,155,ARC00006583,Charpentier libraire-éditeur,1,Charpentier,1,Charpentier,1,Charpentier,0,
6,180,ARC00006475,Calmann-Lévy,2,C Lévy|,2,C Lévy|,1,C Lévy,0,
7,221,ARC00007017,Librairie Académique Perrin et Cie,1,Perrin,1,Perrin,1,Perrin,0,
8,222,ARC00007020,Librairie Académique Perrin et Cie,2,Perrin|Perrin,2,Perrin|Perrin,2,Perrin|Perrin,0,
9,223,ARC00007019,Librairie Académique Perrin et Cie,1,Perrin,1,Perrin,1,Perrin,0,
